### Concesión de acceso dinámico por referencia a grupos de seguridad

*Problema*
Usted tiene un grupo de aplicaciones que actualmente consiste en dos instancias y necesita permitir Secure Shell (SSH) entre ellas. Esto necesita ser configurado de una manera que permita el crecimiento futuro del número de instancias de forma segura y fácil.

*Solución*

**ADVERTENCIA**: Un error común es pensar que simplemente asociando el mismo grupo de seguridad a las ENIs para múltiples instancias EC2, permitirá la comunicación entre ellas (ver Figura).

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0206.png" width="500">

En esta receta, crearemos un grupo de seguridad y asociaremos cada uno a las ENIs de dos instancias EC2. Por último, crearemos una regla de entrada que autorice al grupo de seguridad a llegar a sí mismo en el puerto TCP 22.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0207.png" width="500">

In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.16.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [3]:
# Crear routable
routetable = vpc.create_route_table()
routetable.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT"}])

[ec2.Tag(resource_id='rtb-040ee6910ed1ac89c', key='Name', value='AWSCookbookVPC-RT')]

In [4]:
# Cree una subrede:
subnet = ec2.create_subnet(
    CidrBlock='174.16.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN"}])

[ec2.Tag(resource_id='subnet-0f2ce5796a635e83f', key='Name', value='AWSCookbook-SN')]

In [5]:
# Asociar la tabla de rutas con la subred:
routetable.associate_with_subnet(SubnetId = subnet.id)

ec2.RouteTableAssociation(id='rtbassoc-0798e54c59d166561')

In [6]:
# Crear un grupo de seguridad
security_group = vpc.create_security_group(
    GroupName='AWSCookbook-SG-SSH',
    Description='Allow SSH access',
    VpcId=vpc.id
)

In [7]:
# Añada una regla de entrada al grupo de seguridad que permita el acceso en el puerto TCP 22 desde sí mismo:
ec2_client.authorize_security_group_ingress(
    GroupId=security_group.id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'UserIdGroupPairs': [
                {
                    'Description': 'Allow SSH access',
                    'GroupId': security_group.id
                }
            ]
        } 
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-064194df831060792',
   'GroupId': 'sg-0f440cfe9e9b1bf2d',
   'GroupOwnerId': '967554850238',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'ReferencedGroupInfo': {'GroupId': 'sg-0f440cfe9e9b1bf2d',
    'UserId': '967554850238'},
   'Description': 'Allow SSH access'}],
 'ResponseMetadata': {'RequestId': 'ac379708-f12e-4715-83ca-fb7fb8fa6f63',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ac379708-f12e-4715-83ca-fb7fb8fa6f63',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '903',
   'date': 'Tue, 04 Oct 2022 15:20:04 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

Este tipo de regla de grupo de seguridad se denomina regla de autorreferencia (*self-referencing rule*). Permite el acceso al puerto específico desde el tráfico que se origina en ENIs (no un rango estático de IPs) que tienen este mismo grupo de seguridad adjunto.

#### Crear dos instancias y conectarse mediante el gestor de sesiones SSM

In [8]:
iam = boto3.client('iam')
ssm = boto3.client('ssm')

In [9]:
# Create a role
role_name = 'Cookbook-SSM-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [10]:
# Attach the policy to the role
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': '41f04b10-7b7a-4d18-8de6-d78ae8a54dde',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41f04b10-7b7a-4d18-8de6-d78ae8a54dde',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Tue, 04 Oct 2022 15:20:14 GMT'},
  'RetryAttempts': 0}}

In [11]:
# Create an instance profile
instance_profile_name = 'Cookbook-SSM-Profile'
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)

In [12]:
# Add the role that you created to the instance profile:
iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)

{'ResponseMetadata': {'RequestId': '4131057b-0c2b-4428-9966-bd5b7d8438b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4131057b-0c2b-4428-9966-bd5b7d8438b7',
   'content-type': 'text/xml',
   'content-length': '228',
   'date': 'Tue, 04 Oct 2022 15:20:18 GMT'},
  'RetryAttempts': 0}}

In [13]:
# Enable Dns in VPC
vpc.modify_attribute(EnableDnsSupport={'Value': True})
vpc.modify_attribute(EnableDnsHostnames={'Value': True})

{'ResponseMetadata': {'RequestId': '080b9ddf-6b23-4406-a6aa-e2ccc425dd04',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '080b9ddf-6b23-4406-a6aa-e2ccc425dd04',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Tue, 04 Oct 2022 15:20:22 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [14]:
# Create security group enable HHTPS:
security_group_https = vpc.create_security_group(
    GroupName='AWSCookbook-SG-HTTPS',
    Description='Allow HTTPS access',
    VpcId=vpc.id,
)

ec2_client.authorize_security_group_ingress(
    GroupId=security_group_https.id,
    IpPermissions=[
        {'FromPort': 443,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': vpc.cidr_block,
                    'Description': 'Allow HTTPS access'
                }
            ],
            'ToPort': 443
        }
    ]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-082779ca0835ecb47',
   'GroupId': 'sg-0614443658c044a9f',
   'GroupOwnerId': '967554850238',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '174.16.0.0/16',
   'Description': 'Allow HTTPS access'}],
 'ResponseMetadata': {'RequestId': '25894ff6-6aea-49fb-84a4-07e013a4c8a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '25894ff6-6aea-49fb-84a4-07e013a4c8a0',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '783',
   'date': 'Tue, 04 Oct 2022 15:20:30 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [15]:
# Creates a VPC endpoint for SSM:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssm',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [16]:
# Creates a VPC endpoint for EC2 messages:
vpc_endpoint_ec2msg = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ec2messages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [17]:
# Creates a VPC endpoint for SSMMessages:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssmmessages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

In [30]:
def vpc_endpoint_status(vpc_endpoint_id):
    vpc_endpoint = ec2_client.describe_vpc_endpoints(
        VpcEndpointIds=[vpc_endpoint_id]
    )
    return vpc_endpoint['VpcEndpoints'][0]['State']

In [35]:
print('Waiting for VPC endpoints to be available...')
while vpc_endpoint_status(vpc_endpoint_ssm['VpcEndpoint']['VpcEndpointId']) != 'available':
    time.sleep(5)
    print('SSM endpoint is not available yet...')   
print('SSM endpoint is available')

Waiting for VPC endpoints to be available...
SSM endpoint is available


In [36]:
# Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:
ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [37]:
# Create a key pair
key_name = 'AWSCookbook-KeyPair-Instance'
keypair = ec2.create_key_pair(KeyName=key_name)

In [38]:
# Cear dos instancias EC2 que se puedan conectar a través de SSM
instances = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet.id,
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-SSM-Instance'
                },
            ]
        },
    ]
)

In [39]:
# Esperar a que la instancia esté disponible
instances[-1].wait_until_running()

In [40]:
# Adjunte el grupo de seguridad a las instancias 1 y 2:
for instance in instances:
    instance.modify_attribute(Groups=[security_group.id])

In [41]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-039a49fc7b7c90959


En la terminal:

* Instale la utilidad Ncat:

In [ ]:
sudo yum -y install nc